# 🧹 Data Preprocessing Practice Notebook
This notebook will help you practice key data cleaning and preprocessing steps.

## 1. Load Dataset

In [2]:
import pandas as pd

# Load the synthetic dirty dataset
df = pd.read_csv('dirty_data.csv')
df.head()

,id,name,age,salary,joining_date,gender,city
0,1,Alice,25.0,50000.0,2020-01-10,Male,New York
1,2,Bob,NaN,60000.0,01/15/2021,M,new york
2,3,Charlie,35.0,NaN,"March 5, 2021",male,NEW YORK
3,4,alice,45.0,80000.0,2021-04-12,Female,Los Angeles
4,5,BOB,200.0,1200000.0,12-05-2021,F,los angeles


## 2. Handling Missing Values

In [3]:

# Check missing values
df.isnull().sum()

# Drop missing
df_drop = df.dropna()

# Fill with mean/median/mode
df['age_mean'] = df['age'].fillna(df['age'].mean())
df['age_median'] = df['age'].fillna(df['age'].median())
df['age_mode'] = df['age'].fillna(df['age'].mode()[0])

# KNN Imputation (example with sklearn)
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
df[['age','salary']] = imputer.fit_transform(df[['age','salary']])

# MICE Imputation (using IterativeImputer)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer()
df[['age','salary']] = imp.fit_transform(df[['age','salary']])

df.head()


,id,name,age,salary,joining_date,gender,city,age_mean,age_median,age_mode
0,1,Alice,25.0,5.000000e+04,2020-01-10,Male,New York,25.00,25.0,25.0
1,2,Bob,58.0,6.000000e+04,01/15/2021,M,new york,64.25,34.0,25.0
2,3,Charlie,35.0,6.833333e+04,"March 5, 2021",male,NEW YORK,35.00,35.0,35.0
3,4,alice,45.0,8.000000e+04,2021-04-12,Female,Los Angeles,45.00,45.0,45.0
4,5,BOB,200.0,1.200000e+06,12-05-2021,F,los angeles,200.00,200.0,200.0


## 3. Scaling and Normalization

In [4]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

scaler = MinMaxScaler()
df['salary_minmax'] = scaler.fit_transform(df[['salary']])

scaler = StandardScaler()
df['salary_standard'] = scaler.fit_transform(df[['salary']])

scaler = RobustScaler()
df['salary_robust'] = scaler.fit_transform(df[['salary']])

df[['salary','salary_minmax','salary_standard','salary_robust']].head()


,salary,salary_minmax,salary_standard,salary_robust
0,5.000000e+04,0.000000,-0.375180,-1.307190
1,6.000000e+04,0.008696,-0.345839,-0.522876
2,6.833333e+04,0.015942,-0.321388,0.130719
3,8.000000e+04,0.026087,-0.287156,1.045752
4,1.200000e+06,1.000000,2.999096,88.888889


## 4. Parsing Dates

In [5]:

# Convert messy date formats
df['joining_date_parsed'] = pd.to_datetime(df['joining_date'], errors='coerce', infer_datetime_format=True)
df[['joining_date','joining_date_parsed']]


/var/folders/y5/g1x0tvb16bbf0rjf7219mhmh0000gn/T/ipykernel_47819/1980692542.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['joining_date_parsed'] = pd.to_datetime(df['joining_date'], errors='coerce', infer_datetime_format=True)


,joining_date,joining_date_parsed
0,2020-01-10,2020-01-10
1,01/15/2021,NaT
2,"March 5, 2021",NaT
3,2021-04-12,2021-04-12
4,12-05-2021,NaT
5,NaN,NaT
6,2021/07/20,NaT
7,20210815,NaT
8,2021.09.01,NaT
9,10th Oct 2021,NaT


## 5. Character Encoding & Categorical Conversion

In [6]:

# Standardize gender entries
df['gender_clean'] = df['gender'].str.lower().replace({'m':'male','f':'female','o':'other'})

# One-hot encoding
df_encoded = pd.get_dummies(df, columns=['gender_clean'])
df_encoded.head()


,id,name,age,salary,joining_date,gender,city,age_mean,age_median,age_mode,salary_minmax,salary_standard,salary_robust,joining_date_parsed,gender_clean_female,gender_clean_male,gender_clean_other
0,1,Alice,25.0,5.000000e+04,2020-01-10,Male,New York,25.00,25.0,25.0,0.000000,-0.375180,-1.307190,2020-01-10,False,True,False
1,2,Bob,58.0,6.000000e+04,01/15/2021,M,new york,64.25,34.0,25.0,0.008696,-0.345839,-0.522876,NaT,False,True,False
2,3,Charlie,35.0,6.833333e+04,"March 5, 2021",male,NEW YORK,35.00,35.0,35.0,0.015942,-0.321388,0.130719,NaT,False,True,False
3,4,alice,45.0,8.000000e+04,2021-04-12,Female,Los Angeles,45.00,45.0,45.0,0.026087,-0.287156,1.045752,2021-04-12,True,False,False
4,5,BOB,200.0,1.200000e+06,12-05-2021,F,los angeles,200.00,200.0,200.0,1.000000,2.999096,88.888889,NaT,True,False,False


## 6. Inconsistent Data Entry

In [7]:

# Standardize city names
df['city_clean'] = df['city'].str.lower().str.strip()
df['city_clean'].unique()


array(['new york', 'los angeles', 'chicago', 'miami', nan], dtype=object)

## 7. Dealing with Outliers

In [8]:

import numpy as np

# Z-score method
from scipy import stats
z_scores = np.abs(stats.zscore(df[['age','salary']].dropna()))
outliers = (z_scores > 3)
print("Number of outliers:", outliers.sum())

# IQR method
Q1 = df['salary'].quantile(0.25)
Q3 = df['salary'].quantile(0.75)
IQR = Q3 - Q1
outliers_iqr = df[(df['salary'] < Q1 - 1.5*IQR) | (df['salary'] > Q3 + 1.5*IQR)]
outliers_iqr


Number of outliers: 0


,id,name,age,salary,joining_date,gender,city,age_mean,age_median,age_mode,salary_minmax,salary_standard,salary_robust,joining_date_parsed,gender_clean,city_clean
4,5,BOB,200.0,1200000.0,12-05-2021,F,los angeles,200.0,200.0,200.0,1.0,2.999096,88.888889,NaT,female,los angeles


## 8. Data Deduplication

In [9]:

# Exact duplicates
df_nodup = df.drop_duplicates()

# Fuzzy matching (example with names)
from fuzzywuzzy import process

choices = df['name'].tolist()
process.extract("alice", choices, limit=5)


/opt/homebrew/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[('Alice', 100), ('alice', 100), ('Charlie', 67), ('Eve', 45), ('Ivy', 30)]